# Install & import Libraries and Mount to Google Drive

In [1]:
# !pip install wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import copy
import json
import pandas as pd
from PIL import Image
from time import time
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from torchvision.transforms import Compose, Resize, ToTensor
import wandb

Image.MAX_IMAGE_PIXELS = None

# Import Data and Split Train & Test Sets

In [ ]:
# declare variable
data_file = "/content/drive/MyDrive/P6_data/df_cat.csv"

In [ ]:
# import data
df = pd.read_csv(data_file)[["image", "cat"]]

# label encoding for the targets
lb = LabelEncoder()
df['label'] = lb.fit_transform(df.cat)

# split dataset into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    df.image, df.label, test_size=0.33, random_state=42, stratify=df.label
)

In [ ]:
X_train[:5]

312     c20bf28cd809790c04e78fdaf47c038f.jpg
963     bcb51cec3d290e6a661586d0df30e091.jpg
53      6329ddb46087cb8c09ced95471dd9643.jpg
1039    3b96a152c8fac4421fb67b6190584976.jpg
892     6858fd9265d348803b18667fe43cd054.jpg
Name: image, dtype: object

In [ ]:
y_train[:5]

312     2
963     4
53      2
1039    3
892     4
Name: label, dtype: int64

In [ ]:
X_train.iloc[0]

'c20bf28cd809790c04e78fdaf47c038f.jpg'

In [ ]:
X_train.iloc[1]

'bcb51cec3d290e6a661586d0df30e091.jpg'

# Dataset Preparation
## Dataset

In [ ]:
class BuildDataset(Dataset):
    """
    Load dataset
    """

    def __init__(self, X: pd.Series, y: pd.Series, img_dir: dir, transform=None):
        """
        Initialize the directory containing the images, the labels, and transform images.
        """
        self.X = X
        self.y = y
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        """
        Returns the number of samples in our dataset.
        """
        return len(self.X)

    def __getitem__(self, index: int) -> dict:
        """
        Loads and returns a sample from the dataset at the given index.
        Based on the index, it identifies the image’s location on disk,
        assigns the label accordingly, calls the transform functions on them (if applicable),
        and returns the tensor image and corresponding label in a Python dict.
        """
        image_path = os.path.join(self.img_dir, self.X.iloc[index])
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        sample = {"image": image, "label": self.y.iloc[index]}

        return sample

## Transform

In [ ]:
# declare variables
resize_size = 224
img_dir = "/content/drive/MyDrive/P6_data/images"

In [ ]:
# set transform for train and val sets
transform_train = Compose(
    [
        Resize((resize_size, resize_size)),
        ToTensor(),
    ]
)

transform_val = Compose(
    [
        Resize((resize_size, resize_size)),
        ToTensor(),
    ]
)

In [ ]:
# build dataset for train and val sets
dataset_train = BuildDataset(X_train, y_train, img_dir, transform=transform_train)
dataset_val = BuildDataset(X_val, y_val, img_dir, transform=transform_val)

## Data Loader

In [ ]:
# Set variables
batch_size = 64
num_workers = 4

In [ ]:
# Loads that dataset into the Dataloader and can now iterate through the dataset. 
# "shuffle" = reshuffle the data at every epoch to reduce model overfitting

dataloader_train = DataLoader(
    dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
dataloader_val = DataLoader(
    dataset_val, batch_size=batch_size, shuffle=True, num_workers=num_workers
)

dataloader = {"train": dataloader_train, "val": dataloader_val}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Total number of images in the train dataset
len(dataloader['train'].dataset)

703

In [ ]:
# Number of BATCHs
len(dataloader['train'])

11

In [ ]:
# batch size
data = next(iter(dataloader['train']))
len(data['image'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


64

# Model

In [ ]:
class VGG16(nn.Module):
    def __init__(self, n_classes:int=7):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16(pretrained=True)
        
        # Transfer learning: 
        # Set the features # in the last layer to the features number
        # that we need, i.e., 7 product categories
        num_ftrs = self.vgg16.classifier[-1].in_features
        self.vgg16.classifier[-1] = nn.Linear(num_ftrs, n_classes)

        
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x = self.vgg16(x)
        return x

# Trainer

In [ ]:
def train(model, device, dataloader, optimizer,criterion):
    dataset_size = len(dataloader["train"].dataset)
    batch_losses_train = 0
    batch_corrects_train = 0
    predictions_train = []
    train_labels_list = []

    model.train()
    tbar = tqdm(dataloader["train"], desc="training iterations")
    for batch_idx, data_train in enumerate(tbar):
        train_images = data_train["image"]
        train_labels = data_train["label"]
        train_images = train_images.to(device)
        train_labels = train_labels.to(device)

        ### Train model and compute loss
        outputs = model(train_images)
        preds = outputs.data.max(1)[1].squeeze()
        losses = criterion(outputs, train_labels)
        batch_losses_train += losses.item()
        batch_corrects_train += torch.sum(preds == train_labels).cpu()

        tbar.set_description(
            f"Average train loss: {(batch_losses_train / (batch_idx + 1)):.3f}"
        )

        predictions_train += preds.cpu().numpy().tolist()
        train_labels_list += train_labels.cpu().numpy().tolist()

        ### Backpropagation
        # Sets the gradients of all optimized 'torch.Tensor's to zero.
        optimizer.zero_grad()
        # Compute gradient
        losses.backward()
        # Gradient Descent
        optimizer.step()

    epoch_loss_train = batch_losses_train / dataset_size
    epoch_acc_train = batch_corrects_train / dataset_size
    F1_score = f1_score(predictions_train, train_labels_list, average='weighted')
    tqdm.write(
        f"Training >>> Loss : {epoch_loss_train:.5f}, Acc : {epoch_acc_train:.4%}"
    )

    wandb.log(
        {
            "Training Accuracy": epoch_acc_train * 100.0,
            "Training Loss": epoch_loss_train,
            "F1 training score": F1_score,
        }
    )

    return epoch_loss_train, epoch_acc_train

In [ ]:
def validate(model, device, dataloader, classes, criterion):
    dataset_size = len(dataloader["val"].dataset)
    batch_losses_val = 0
    batch_corrects_val = 0
    predictions_val = []
    val_labels_list = []
    example_images = []

    model.eval()
    vbar = tqdm(dataloader["val"], desc="validation iterations")
    for batch_idx, data_val in enumerate(vbar):
        val_images = data_val["image"]
        val_labels = data_val["label"]
        val_images = val_images.to(device)
        val_labels = val_labels.to(device)

        ### Evaluation model and compute loss
        with torch.no_grad():
            outputs = model(val_images)
        preds = outputs.data.max(1)[1].squeeze()
        losses = criterion(outputs, val_labels)

        batch_losses_val += losses.item()
        batch_corrects_val += torch.sum(preds == val_labels).cpu()
        vbar.set_description(
            f"Average val loss: {(batch_losses_val / (batch_idx + 1)):.3f}"
        )

        predictions_val += preds.cpu().numpy().tolist()
        val_labels_list += val_labels.cpu().numpy().tolist()

        example_images.append(
            wandb.Image(
                val_images[0],
                caption=f"Pred: {classes[preds[0].item()]} Truth: {classes[val_labels[0].item()]}",
            )
        )

    epoch_loss_val = batch_losses_val / dataset_size
    epoch_acc_val = batch_corrects_val / dataset_size
    F1_score = f1_score(predictions_val, val_labels_list, average='weighted')
    tqdm.write(
        f"Validation >>> Loss : {epoch_loss_val:.5f}, Acc : {epoch_acc_val:.4%}\n"
    )

    wandb.log(
        {
            "Validation Accuracy": epoch_acc_val * 100.0,
            "Validation Loss": epoch_loss_val,
            "Validation exmaples": example_images,
            "F1 validation score": F1_score,
        }
    )

    return epoch_loss_val, epoch_acc_val

In [ ]:
def trainer(config, model, device, dataloader, optimizer, criterion):

    ep_losses_train = []
    ep_losses_val = []
    ep_acc_train = []
    ep_acc_val = []
    classes = {
        0: "Baby Care",
        1: "Beauty and Personal Care",
        2: "Computers",
        3: "Home Decor & Festive Needs",
        4: "Home Furnishing",
        5: "Kitchen & Dining",
        6: "Watches",
    }

    best_acc_val = 0

    tic = time()
    # start training model for each set of hyper-parameters
    for epoch in tqdm(range(config.epochs), desc="epochs"):
        tqdm.write(f"epoch: {epoch+1}")
        loss_train, acc_train = train(model, device, dataloader, optimizer, criterion)
        loss_val, acc_val = validate(model, device, dataloader, classes, criterion)

        ep_losses_train.append(loss_train)
        ep_acc_train.append(acc_train)

        ep_losses_val.append(loss_val)
        ep_acc_val.append(acc_val)

        # save best models
        if acc_val >= best_acc_val:
            best_epoch = epoch
            best_acc_train = acc_train
            best_acc_val = acc_val
            best_model_path = f"/content/drive/MyDrive/P6_data/models/{model.__class__.__name__}_acc={acc_val}_lr={config.lr}_wd={config.weight_decay}_do={config.dropout}.pth"
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), best_model_path)

    tac = time()
    print("Training finished")
    print("===========================================================================")
    print(
        f"Time elapsed: {tac - tic:.2f} s, Best epoch: {best_epoch}, Best accuracy: {best_acc_val:.2%}"
    )
    return best_acc_train, best_acc_val

In [ ]:
def experiment():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {"epochs": 16, "batch_size": 64, "lr": 1e-2, "momentum": 0.4}

    # Initialize a new wandb run
    wandb.init(config=config_defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    # Get GPU, or else CPU, device for training.
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"Using {device} device")

    # MODEL
    model = VGG16()
    model.to(device)

    # optimizer
    optimizer = optim.Adam(
        model.parameters(), lr=config.lr, weight_decay=config.weight_decay
    )

    # criterion
    criterion = torch.nn.CrossEntropyLoss()

    # dataloader
    dataloader_train = DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers
    )
    dataloader_val = DataLoader(
        dataset_val, batch_size=batch_size, shuffle=True, num_workers=num_workers
    )
    dataloader = {"train": dataloader_train, "val": dataloader_val}

    # train model
    best_acc_train, best_acc_val = trainer(
        config,
        model,
        device,
        dataloader,
        optimizer,
        criterion,
    )

# Weights and Biases Sweep Initialisation

In [ ]:
# configure the sweep – specify the parameters to search through, the search strategy, optimization metric etc.
sweep_config = {
    "method": "bayes",
    "metric": {"name": "Validation Loss", "goal": "minimize"},
    "parameters": {
        'dropout': {
           'values': [0, 0.1, 0.2, 0.3, 0.4]
        },
        'lr': {
            'values': [1e-4, 1e-3, 1e-2, 1e-1]
        },
        'weight_decay': {
            'values': [1e-6, 1e-5, 1e-4]
        },
    },
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="yws", project=f"VGG16_{len(dataloader['train'].dataset)}images_sweeps")

Create sweep with ID: bj08v0z1
Sweep URL: https://wandb.ai/yws/VGG16_703images_sweeps/sweeps/bj08v0z1


# Experiment

In [ ]:
wandb.agent(sweep_id, experiment)

wandb: Agent Starting Run: dohgbp7t with config:
wandb: 	dropout: 0.2
wandb: 	lr: 0.01
wandb: 	weight_decay: 1e-06


Using cuda:0 device


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epochs:   0%|          | 0/16 [00:00<?, ?it/s]

epoch: 1


...

validation iterations:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validation >>> Loss : 0.03367, Acc : 14.1210%

epoch: 14


training iterations:   0%|          | 0/11 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
